In [2]:
!pip install h3
!pip install keplergl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 4.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 41.1 MB/s 
     |████████████████████████████████| 6.3 MB 52.2 MB/s 
     |████████████████████████████████| 16.7 MB 168 kB/s 
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922344 sha256=35e27d8ea7cf8b970bc640c84b0b91f9de5416c8fb995dec97fb7bfe65e6bbde
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


In [17]:
import sys
import pandas as pd
import os
import numpy as np
from keplergl import KeplerGl
from tqdm.auto import tqdm

import pickle
from h3 import h3

def load_pickle(file_path):
    with open(file_path, 'rb') as handle:
        return pickle.load(handle)
        
def save_pickle(obj, filepath): 
    with open(filepath, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/Alfa-data

/content/drive/MyDrive/Colab Notebooks/Alfa-data


In [6]:
df_roads = pd.read_csv("roads_dataset.csv")
df_transport = pd.read_csv("routes_dataset.csv")
df_population = pd.read_csv("rosstat_population_all_cities.csv")
df_isochrones = pd.read_csv("isochrones_walk_dataset.csv")
df_stops = pd.read_csv("osm_stops.csv")
df_companies = pd.read_csv("osm_amenity.csv")
df_target = pd.read_csv("target_hakaton_spb.csv", encoding='cp1251')

In [7]:
df_target.rename(columns = {'lat_h3' : 'lat', 'lon_h3' : 'lon'}, inplace = True)

# Найдём район для каждого банкомата

## Преобразуем изохроны для удобства

In [8]:
isochrones=df_isochrones.copy()
isochrones['walk_15min']=isochrones['walk_15min'].str.replace(',','')
isochrones['walk_15min']=isochrones['walk_15min'].str.replace('(','')
isochrones['walk_15min']=isochrones['walk_15min'].str.replace(')','')
isochrones['walk_15min']=isochrones['walk_15min'].str.replace('(','')
isochrones['walk_15min']=isochrones['walk_15min'].str.replace('POLYGON','')

for i, row in isochrones.iterrows(): # Сплитим по координатам, нечётная - lat, чётная - lon, и создаём списки
  isochrones['walk_15min'][i] = isochrones['walk_15min'][i].split(" ")[1:]

for i, row in isochrones.iterrows(): # Преобразовываем строки в числа
  isochrones['walk_15min'][i] = [float(numeric_string) for numeric_string in isochrones['walk_15min'][i]]

isochrones.insert(2, "y_of_dots", np.nan) # Добавим колонку координат y
isochrones.insert(2, "x_of_dots", np.nan) # Добавим колонку координат x

isochrones_normal_list=isochrones.copy()

for i, row in isochrones_normal_list.iterrows(): # Заполним колонки, которые добавили на предыдущем шаге
  isochrones_normal_list['x_of_dots'][i]=np.array(isochrones_normal_list['walk_15min'][i][::2], dtype=object)
  isochrones_normal_list['y_of_dots'][i]=np.array(isochrones_normal_list['walk_15min'][i][1::2], dtype=object)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWit

In [11]:
def inPolygon(x, y, xp, yp):
    c=0
    for i in range(len(xp)):
        if (((yp[i]<=y and y<yp[i-1]) or (yp[i-1]<=y and y<yp[i])) and 
            (x > (xp[i-1] - xp[i]) * (y - yp[i]) / (yp[i-1] - yp[i]) + xp[i])): c = 1 - c    
    return c

In [12]:
df = df_target.merge(df_isochrones[['walk_15min', 'geo_h3_10']], how='left', on='geo_h3_10')

In [14]:
df = df[~df.duplicated(subset=['geo_h3_10'], keep='last')] 

In [15]:
def distance(x1, y1, x2, y2):
  return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))

In [18]:
for j, row_ in tqdm(df[df['walk_15min'].isna()].iterrows()): # Подберём подходящий район для пропущенных значений 
  print(f'Num of df = {j}')
  tmp = isochrones_normal_list[isochrones_normal_list['city'] == row_.gorod]
  for i, row in tmp.iterrows():
    mask = df['geo_h3_10'] == row_.geo_h3_10 
    if df["walk_15min"][j] is np.nan:  # Сделаем маску
        df.loc[mask, 'walk_15min'] = str(row['walk_15min'])
        df.loc[mask, 'distance'] = distance(row['lat'], row['lon'], df["lat"][j], df["lon"][j])
    else:
      if len(df.loc[mask, 'distance'].values) == 1:
        if distance(row['lat'], row['lon'], row_["lat"], row_["lon"]) < df.loc[mask, 'distance'].values:
          df.loc[mask, 'walk_15min'] = str(row['walk_15min'])
          df.loc[mask, 'distance'] = distance(row['lat'], row['lon'], df["lat"][j], df["lon"][j]) 
      else:
        if distance(row['lat'], row['lon'], row_["lat"], row_["lon"]) < df.loc[mask, 'distance'].values[0]:
          df.loc[mask, 'walk_15min'] = str(row['walk_15min'])
          df.loc[mask, 'distance'] = distance(row['lat'], row['lon'], df["lat"][j], df["lon"][j]) 

0it [00:00, ?it/s]

Num of df = 9


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


Num of df = 16
Num of df = 21
Num of df = 22
Num of df = 57
Num of df = 58
Num of df = 66
Num of df = 67
Num of df = 68
Num of df = 80
Num of df = 105
Num of df = 108
Num of df = 115
Num of df = 116
Num of df = 122
Num of df = 126
Num of df = 134
Num of df = 138
Num of df = 140
Num of df = 145
Num of df = 146
Num of df = 157
Num of df = 158
Num of df = 161
Num of df = 162
Num of df = 163
Num of df = 164
Num of df = 168
Num of df = 173
Num of df = 174
Num of df = 175
Num of df = 176
Num of df = 178
Num of df = 179
Num of df = 193
Num of df = 194
Num of df = 200
Num of df = 204
Num of df = 206
Num of df = 207
Num of df = 211
Num of df = 212
Num of df = 213
Num of df = 220
Num of df = 222
Num of df = 233
Num of df = 238
Num of df = 239
Num of df = 245
Num of df = 247
Num of df = 255
Num of df = 259
Num of df = 269
Num of df = 270
Num of df = 272
Num of df = 276
Num of df = 279
Num of df = 280
Num of df = 281
Num of df = 282
Num of df = 283
Num of df = 284
Num of df = 285
Num of df = 287
N

Работа с транспортом

In [254]:
df_new = df.copy()
df_new.insert(3, "transport_count", 0)

In [255]:
h3_result = lambda id_str: h3.h3_to_geo_boundary(id_str) if h3.h3_is_valid(id_str) else None

for i, row in df_new.iterrows():
  la, lo = df_stops['lat'][i], df_stops['lon'][i]
  hash = h3.geo_to_h3(
    lat=la,
    lng=lo,
    resolution=10
  )

  if list(df_new['geo_h3_10']).count(hash):
      df_new['transport_count'][i] += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [256]:
line = df_transport.copy()
line['geometry']=line['geometry'].str.replace(',','')
line['geometry']=line['geometry'].str.replace('(','')
line['geometry']=line['geometry'].str.replace(')','')
line['geometry']=line['geometry'].str.replace('LINESTRING','')

for i, row in df_new.iterrows():
  line['geometry'][i] = line['geometry'][i].split(" ")[1:]
  line['geometry'][i] = [float(numeric_string) for numeric_string in line['geometry'][i]]
  tmp = ''

  for j in range(len(line['geometry'][i])-1):
    la, lo = line['geometry'][i][j], line['geometry'][i][j+1]
    hash = h3.geo_to_h3(
      lat=la,
      lng=lo,
      resolution=10
    )
    
    if list(df_new['geo_h3_10']).count(hash) and tmp != hash: # df_new['transport_count'][i] != 0:
      df_new['transport_count'][i] += 1
      tmp = hash

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Присоединяем фичи

In [257]:
df_wit_features = df_new.merge(df_companies[['geo_h3_10','Автозапчасти для иномарок',
       'Авторемонт и техобслуживание (СТО)', 'Алкогольные напитки', 'Аптеки',
       'Банки', 'Быстрое питание', 'Доставка готовых блюд', 'Женская одежда',
       'Кафе', 'Косметика / Парфюмерия', 'Ногтевые студии', 'Овощи / Фрукты',
       'Парикмахерские', 'Платёжные терминалы', 'Постаматы',
       'Продуктовые магазины', 'Пункты выдачи интернет-заказов', 'Рестораны',
       'Страхование', 'Супермаркеты', 'Цветы', 'Шиномонтаж']], how='left', on='geo_h3_10')

In [258]:
df_wit_features = df_wit_features.merge(df_population[['geo_h3_10', 'population']], how='left', on='geo_h3_10')

In [259]:
df_with_features_save = df_wit_features.copy()

In [260]:
df_wit_features = df_with_features_save.copy()

Заполняем population

In [261]:
mean = df_wit_features["population"].mean()
std = df_wit_features["population"].std()
is_null = df_wit_features["population"].isnull().sum()
rand_population = np.random.randint(mean - std, mean + std, size = is_null)
population_slice = df_wit_features["population"].copy()
population_slice[np.isnan(population_slice)] = rand_population
df_wit_features["population"] = population_slice
df_wit_features["population"] = df_wit_features["population"].astype(float)

In [262]:
df_wit_features

,gorod,geo_h3_10,lat,transport_count,lon,atm_category,target,atm_cnt,walk_15min,distance,...,Платёжные терминалы,Постаматы,Продуктовые магазины,Пункты выдачи интернет-заказов,Рестораны,Страхование,Супермаркеты,Цветы,Шиномонтаж,population
0,Екатеринбург,8a10c2283357fff,56.865861,3,60.532224,category4,95.333333,1,"POLYGON ((60.530224 56.874333, 60.528586 56.87...",NaN,...,7.0,NaN,NaN,1.0,NaN,1.0,6.0,3.0,NaN,798.0
1,Екатеринбург,8a10c2288087fff,56.885391,2,60.588203,category3,30652.666667,4,"POLYGON ((60.593203 56.895636, 60.591203 56.89...",NaN,...,NaN,1.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,667.0
2,Екатеринбург,8a10c228820ffff,56.891648,0,60.605399,category2,1162.166667,1,"POLYGON ((60.611399 56.900068, 60.609399 56.90...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,528.0
3,Екатеринбург,8a10c2288adffff,56.895039,2,60.598777,category1,2961.333333,1,"POLYGON ((60.603777 56.904214, 60.602777 56.90...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,675.0
4,Екатеринбург,8a10c228944ffff,56.896206,0,60.625911,category1,2963.333333,1,"POLYGON ((60.624911 56.905507, 60.618911 56.90...",NaN,...,1.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,321.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,Санкт-Петербург,8a1106ab618ffff,59.986278,0,30.217541,category1,10523.666667,1,"POLYGON ((30.219541 59.994434, 30.218541 59.99...",NaN,...,6.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,580.0
491,Санкт-Петербург,8a1106ab67a7fff,59.983488,0,30.227538,category1,2535.833333,1,"POLYGON ((30.232538 59.992805, 30.231538 59.99...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245.0
492,Санкт-Петербург,8a1106ab6807fff,59.990519,0,30.205250,category4,94.500000,1,"POLYGON ((30.20325 60.00073, 30.202425 60.0003...",NaN,...,4.0,NaN,NaN,NaN,4.0,NaN,2.0,1.0,NaN,605.0
493,Санкт-Петербург,8a1106ab6ae7fff,59.996443,0,30.219656,category2,489.500000,1,"POLYGON ((30.225656 60.007665, 30.223656 60.00...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,558.0


Удаляем вспомогательные поля disctance, lat, lon

In [264]:
h3_column = df_wit_features['geo_h3_10']

In [265]:
df_wit_features.drop(['distance', 'lat', 'lon', 'geo_h3_10', 'walk_15min'], axis=1, inplace=True)

In [266]:
df_wit_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   gorod                               495 non-null    object 
 1   transport_count                     495 non-null    int64  
 2   atm_category                        495 non-null    object 
 3   target                              495 non-null    float64
 4   atm_cnt                             495 non-null    int64  
 5   Автозапчасти для иномарок           22 non-null     float64
 6   Авторемонт и техобслуживание (СТО)  15 non-null     float64
 7   Алкогольные напитки                 64 non-null     float64
 8   Аптеки                              179 non-null    float64
 9   Банки                               122 non-null    float64
 10  Быстрое питание                     160 non-null    float64
 11  Доставка готовых блюд               61 non-nu

In [267]:
df_wit_features_save_2 = df_wit_features.copy()

In [268]:
df_wit_features = df_wit_features_save_2.copy()

Заполним Nan-ы

In [269]:
for i in df_wit_features.columns:
  df_wit_features[i] = df_wit_features[i].fillna(0)

In [270]:
df_wit_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   gorod                               495 non-null    object 
 1   transport_count                     495 non-null    int64  
 2   atm_category                        495 non-null    object 
 3   target                              495 non-null    float64
 4   atm_cnt                             495 non-null    int64  
 5   Автозапчасти для иномарок           495 non-null    float64
 6   Авторемонт и техобслуживание (СТО)  495 non-null    float64
 7   Алкогольные напитки                 495 non-null    float64
 8   Аптеки                              495 non-null    float64
 9   Банки                               495 non-null    float64
 10  Быстрое питание                     495 non-null    float64
 11  Доставка готовых блюд               495 non-n

In [271]:
prepared_data = df_wit_features.copy()

Сохраним csv

In [272]:
Features = prepared_data.copy()

Get_dummies for категориальных фичей 

In [273]:
#Преобразуем категориальные признаки
Features_dum = pd.get_dummies(Features, drop_first=True)
print(Features_dum.shape)
Features_dum.head()

(495, 32)


,transport_count,target,atm_cnt,Автозапчасти для иномарок,Авторемонт и техобслуживание (СТО),Алкогольные напитки,Аптеки,Банки,Быстрое питание,Доставка готовых блюд,...,Супермаркеты,Цветы,Шиномонтаж,population,gorod_Нижний Новгород,gorod_Новосибирск,gorod_Санкт-Петербург,atm_category_category2,atm_category_category3,atm_category_category4
0,3,95.333333,1,0.0,0.0,0.0,2.0,0.0,3.0,0.0,...,6.0,3.0,0.0,798.0,0,0,0,0,0,1
1,2,30652.666667,4,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,667.0,0,0,0,0,1,0
2,0,1162.166667,1,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,528.0,0,0,0,1,0,0
3,2,2961.333333,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,675.0,0,0,0,0,0,0
4,0,2963.333333,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,321.0,0,0,0,0,0,0


# Разделим данные на train и validation

In [274]:
from sklearn import preprocessing

In [275]:
from sklearn.model_selection import train_test_split

In [303]:
train, test = train_test_split(Features_dum, test_size=0.1)

In [304]:
X_train = train.drop(['target'], axis=1)
y_train = train['target'].copy()

X_test = test.drop(['target'], axis=1)
y_test = test['target'].copy()

### Нормализация признаки

In [305]:
# Нормируем признаки
normalizer = preprocessing.MinMaxScaler()
X_train_values = normalizer.fit_transform(X_train.values)

In [306]:
X_test_values = normalizer.transform(X_test.values)

# Обучение XGBoost

In [307]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [308]:
model_xgb = xgb.XGBRegressor()

In [309]:
model_xgb.fit(X_train_values, y_train)

[11:31:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [310]:
print(model_xgb.score(X_test_values, y_test))

0.8276687460293773
